In [ ]:
!git clone https://github.com/huggingface/diffusers.git
%cd diffusers
!pip install .

In [1]:
%cd diffusers
!pip install .

/opt/conda/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/home/jupyter/diffusers
Processing /home/jupyter/diffusers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached huggingface_hub-0.26.2-py3-none-any.whl.metadata (13 kB)
  Using cached regex-2024.11.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (40 kB)
  Using cached safetensors-0.4.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
Using cached huggingface_hub-0.26.2-py3-none-any.whl (447 kB)
Using cached regex-2024.11.6-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (781 kB)
Using cached safetensors-0.4.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (435 kB)
  Created wheel for diffusers: filename=diffusers-0.32.0.dev0-py3-none-any.whl size=2977364 sha256=766de093a1883dbbd94a7c7c69c99d37c120ab524403b6f541574c39c36656f2
  Stored in directory: /tmp/pip-ephem-wheel-cache-rnsfv3_0/wheels/57/60/f2/fe2861f83705358f706e

In [2]:
%cd examples/text_to_image
!pip install -r requirements.txt

/home/jupyter/diffusers/examples/text_to_image
  Using cached accelerate-1.1.1-py3-none-any.whl.metadata (19 kB)
  Using cached transformers-4.46.3-py3-none-any.whl.metadata (44 kB)
  Using cached datasets-3.1.0-py3-none-any.whl.metadata (20 kB)
  Using cached ftfy-6.3.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached tensorboard-2.18.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached peft-0.7.0-py3-none-any.whl.metadata (25 kB)
  Using cached tokenizers-0.20.3-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached pyarrow-18.1.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached multiprocess-0.70.16-py310-none-any.whl.metadata (7.2 kB)
  Using cached fsspec-2024.9.0-py3-none-any.whl.metadata (11 kB)
  Using cached Markdown-3.7-py3-none-any.whl.metadata (7.0 kB)
  Using cached tensorboard_data_server-0.7.2-py3-none-manylinux_2_31_x86_64.whl.metadata (1.1 kB)
  Using c

In [ ]:
!ls /home/jupyter/dataset

In [ ]:
!gsutil -m cp -r gs://text-to-image-429503/flickr8k/Images/* /home/jupyter/dataset

In [ ]:
import os
import json
import shutil

# Paths
images_folder = "/home/jupyter/dataset"  
metadata_path=os.path.join(images_folder, "metadata.jsonl")
output_folder = "/home/jupyter/shortened_dataset"
shortened_metadata_path = os.path.join(output_folder, "metadata.jsonl")

os.makedirs(output_folder, exist_ok=True)

# Processing
unique_images = set()
shortened_metadata = []

with open(metadata_path, "r") as metadata_file:
    for line in metadata_file:
        entry = json.loads(line.strip())
        file_name = entry["file_name"]

        # Add to unique images until 200 unique file_names are reached
        if len(unique_images) < 200:
            if file_name not in unique_images:
                unique_images.add(file_name)
                # Copy the corresponding image to the new folder
                src_path = os.path.join(images_folder, file_name)
                dst_path = os.path.join(output_folder, file_name)
                if os.path.exists(src_path):
                    shutil.copy(src_path, dst_path)
                else:
                    print(f"Warning: Image file {src_path} not found!")

            # Add the entry to the new metadata
            shortened_metadata.append(entry)

# Write the shortened metadata to a new JSONL file
with open(shortened_metadata_path, "w") as output_file:
    for entry in shortened_metadata:
        output_file.write(json.dumps(entry) + "\n")

print("Dataset culling complete. Shortened metadata and images stored in /shortened_dataset.")


In [4]:
import torch
import torch_xla
import torch_xla.core.xla_model as xm

dev = xm.xla_device()
t1 = torch.ones(3, 3, device=dev)
print(t1)

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]], device='xla:0')


In [8]:
!rm -r /home/jupyter/model
!mkdir /home/jupyter/model

In [9]:
!accelerate launch train_text_to_image.py \
  --pretrained_model_name_or_path=CompVis/stable-diffusion-v1-4  \
  --train_data_dir=/home/jupyter/shortened_dataset  \
  --use_ema \
  --resolution=512 --center_crop --random_flip \
  --train_batch_size=48 \
  --gradient_accumulation_steps=1 \
  --gradient_checkpointing \
  --mixed_precision=fp16 \
  --max_train_steps=500 \
  --learning_rate=5e-06 \
  --max_grad_norm=1 \
  --lr_scheduler="linear" \
  --lr_warmup_steps=0\
  --output_dir=/home/jupyter/model

/opt/conda/lib/python3.10/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/opt/conda/lib/python3.10/site-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https:/

In [16]:
%cd ../../../

/home/jupyter


/opt/conda/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [17]:
!ls

SDFinetune.ipynb  diffusers  new-workspace.jupyterlab-workspace  src
dataset		  model      shortened_dataset			 tutorials


In [21]:
from diffusers import StableDiffusionPipeline

model_path = "model"
pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float16)
pipe.to("cuda")

image = pipe(prompt="Dave").images[0]
image.save("test.png")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

In [22]:
!gsutil -m cp -r model gs://text-to-image-model-429503

Copying file://model/model_index.json [Content-Type=application/json]...
Copying file://model/logs/text2image-fine-tune/events.out.tfevents.1732728687.5d19749c37f3.259.0 [Content-Type=application/octet-stream]...
Copying file://model/logs/text2image-fine-tune/1732728687.324261/hparams.yml [Content-Type=application/octet-stream]...
Copying file://model/logs/text2image-fine-tune/1732728687.3225884/events.out.tfevents.1732728687.5d19749c37f3.259.1 [Content-Type=application/octet-stream]...
Copying file://model/unet/config.json [Content-Type=application/json]...        
Copying file://model/unet/diffusion_pytorch_model.safetensors [Content-Type=application/octet-stream]...
Copying file://model/tokenizer/vocab.json [Content-Type=application/json]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
